<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Изучение-данных" data-toc-modified-id="Изучение-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Изучение данных</a></span><ul class="toc-item"><li><span><a href="#Выводы" data-toc-modified-id="Выводы-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Выводы</a></span></li></ul></li><li><span><a href="#Разбейте-данные-на-выборки" data-toc-modified-id="Разбейте-данные-на-выборки-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Разбейте данные на выборки</a></span></li><li><span><a href="#Исследование-моделей" data-toc-modified-id="Исследование-моделей-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Исследование моделей</a></span><ul class="toc-item"><li><span><a href="#Решающее-дерево" data-toc-modified-id="Решающее-дерево-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Решающее дерево</a></span></li><li><span><a href="#Случайный-лес" data-toc-modified-id="Случайный-лес-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Случайный лес</a></span></li><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li><li><span><a href="#Выводы-по-исследованию-моделей" data-toc-modified-id="Выводы-по-исследованию-моделей-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Выводы по исследованию моделей</a></span></li></ul></li><li><span><a href="#Проверка-лучшей-модели-на-тестовой-выборке" data-toc-modified-id="Проверка-лучшей-модели-на-тестовой-выборке-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка лучшей модели на тестовой выборке</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Проверка-модели-на-адекватность" data-toc-modified-id="Проверка-модели-на-адекватность-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Проверка модели на адекватность</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Итоговый-вывод" data-toc-modified-id="Итоговый-вывод-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Итоговый вывод</a></span></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li></ul></div>

# Введение в машинное обучение 
# Проект: Выбор подходящего тарифа сотовой связи
# Описание проекта
Оператор мобильной связи «Мегалайн» выяснил, что многие клиенты пользуются архивными тарифами и хочет построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

В распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных уже сделана.

Нужно построить несколько моделей и выбрать одну с максимально большим значением `accuracy`. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75 и проверить лучшую модель на тестовой выборке.

## Изучение данных

Импортируем нужные библиотеки и функции.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score,  f1_score

Загрузим датасет `.csv` и сохраним его в переменной df.

In [2]:
try:    
    df = pd.read_csv('D:/notebook/project_6/users_behavior.csv')   
except:
    df = pd.read_csv('/datasets/users_behavior.csv')
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


# Описание данных
Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:

* `сalls` — количество звонков,
* `minutes` — суммарная длительность звонков в минутах,
* `messages` — количество sms-сообщений,
* `mb_used` — израсходованный интернет-трафик в Мб,
* `is_ultra` — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [4]:
df.duplicated().sum() # вычислим количество грубых дубликатов

0

In [5]:
df.describe() # посмотрим статистические характеристики данных.

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


### Выводы
* Таблица данных содержит 3214 объектов, пропусков нет.
* Сильного перекоса в распределениях признаков не наблюдается, медианы и средние значения находятся рядом по всем параметрам.

## Разбейте данные на выборки

Выделим из датасета:
* признаки: столбцы `calls, minutes, messages, mb_used`
* целевой признак: столбец `is_ultra`

In [6]:
features = df.drop('is_ultra', axis=1)
target = df['is_ultra']

Разделим данные на обучающую - 60% , валидационную - 20 % и тестовую - 20 % выборки.

In [7]:
X, X_test, y, y_test = train_test_split(features, target, test_size=0.2, random_state=123)    # выделим тестовую выборку
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=123) # выделим обучающую и валидационную выборки

print('Размер обучающей выборки признаков:            ', X_train.shape[0])
print('Размер обучающей выборки целевого признака:    ', y_train.shape[0])
print('Размер валидационной выборки признаков:        ', X_valid.shape[0])
print('Размер валидационной выборки целевого признака:', y_valid.shape[0])
print('Размер тестовой выборки признаков:             ', X_test.shape[0])
print('Размер тестовой выборки целевого признака:     ', y_test.shape[0])

Размер обучающей выборки признаков:             1928
Размер обучающей выборки целевого признака:     1928
Размер валидационной выборки признаков:         643
Размер валидационной выборки целевого признака: 643
Размер тестовой выборки признаков:              643
Размер тестовой выборки целевого признака:      643


## Исследование моделей

### Решающее дерево

Применим модель `DecisionTreeClassifier`

Выделим гиперпараметры для настройки модели:
* `max_depth` - максимальная глубина древа
* `min_samples_split` - минимальное количество объектов в узле
* `min_samples_leaf` - минимальное количество объектов в листе
* `criterion` ("gini" или "entropy") - функция измерения качества разделения

Точность модели `accuracy`.

In [8]:
%%time 
# время выполнения блока.

best_params_dtc = []  # список лучших параметров
best_accuracy_dtc = 0 # лучшая точность

for max_depth in range(2,10):                    # перебор гиперпараметров по заданным диапазонам
    for min_samples_split in range(2,6):
        for min_samples_leaf in range(1,10):
            for criterion in ['gini', 'entropy']:
                # вызов модели 
                model_dtc = DecisionTreeClassifier(
                random_state=784, max_depth=max_depth, min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf, 
                criterion=criterion)
                
                model_dtc.fit(X_train, y_train)             # обучение модели на тренировочной выборке                
                predictions_dtc = model_dtc.predict(X_valid)    # поиск предсказаний модели на валидационной выборке                
                accuracy_dtc = accuracy_score(y_valid, predictions_dtc)# вычисление точности модели методом 
                
                if accuracy_dtc > best_accuracy_dtc:          # если текущее значение точности выше предыдущего лучшего значения,
                    best_params_dtc = [max_depth, min_samples_split, min_samples_leaf, criterion]
                    best_accuracy_dtc = accuracy_dtc          # сохраняем параметры модели и текущую точность            


Wall time: 2.88 s


Выделим параметры лучшей модели обучающего леса.

In [9]:
print('Максимальная точность:', best_accuracy_dtc)
print('max_depth:            ', best_params_dtc[0])
print('min_samples_split:    ', best_params_dtc[1])
print('min_samples_leaf:     ', best_params_dtc[2])
print('criterion:            ', best_params_dtc[3])

Максимальная точность: 0.7947122861586314
max_depth:             6
min_samples_split:     2
min_samples_leaf:      5
criterion:             entropy


Результат, удовлетворяющий условие.

### Случайный лес

Применим модель `RandomForestClassifier`
Выделим гиперпараметры для настройки модели:
* n_estimators - количество деревьев
* max_depth - максимальная глубина древа
* min_samples_split - минимальное количество объектов в узле
* min_samples_leaf - минимальное количество объектов в листе
* criterion ("gini" или "entropy") - функция измерения качества разделения

Точность модели `accuracy`.

In [10]:
%%time 
# время выполнения блока.

best_params_rfc = []  # список лучших параметров
best_accuracy_rfc = 0 # лучшая точность

for n_estimators in (1,20):                              # перебор гиперпараметров по заданным диапазонам
    for max_depth in range(2,10):
        for min_samples_split in range(2,10):
            for min_samples_leaf in range(1,8):
                for criterion in ['gini', 'entropy']:
                    # вызовем модель
                    model_rfc = RandomForestClassifier(
                    random_state=784, n_estimators=n_estimators, max_depth=max_depth, min_samples_split=min_samples_split,
                    min_samples_leaf=min_samples_leaf, criterion=criterion)
                    
                    model_rfc.fit(X_train, y_train)             # обучение модели на тренировочной выборке                    
                    predictions_rfc = model_rfc.predict(X_valid)    # поиск предсказаний модели на валидационной выбоке                    
                    accuracy_rfc = accuracy_score(y_valid, predictions_rfc)# вычисление точности модели                    
                    
                    if accuracy_rfc > best_accuracy_rfc:       # если текущее значение точности выше предыдущего лучшего значения, 
                        best_params_rfc = [n_estimators, max_depth, min_samples_split, min_samples_leaf, criterion]
                        best_accuracy_rfc = accuracy_rfc       # сохраняем параметры модели и текущую точность

Wall time: 42.5 s


Выделим параметры лучшей модели случайного леса.

In [11]:
print('Максимальная точность:', best_accuracy_rfc)
print('n_estimators:         ', best_params_rfc[0])
print('max_depth:            ', best_params_rfc[1])
print('min_samples_split:    ', best_params_rfc[2])
print('min_samples_leaf:     ', best_params_rfc[3])
print('criterion:            ', best_params_rfc[4])

Максимальная точность: 0.8087091757387247
n_estimators:          20
max_depth:             9
min_samples_split:     4
min_samples_leaf:      1
criterion:             entropy


Результат, удовлетворяющий условие.

### Логистическая регрессия
Вызовем модель `LogisticRegression`


In [12]:
%%time

# время выполнения блока

best_max_iter_lr = 0  # лучшее число итераций
best_accuracy_lr = 0  # лучшая точность

for max_iter in range(1,1000):
    model_lr = LogisticRegression(random_state=784, max_iter=max_iter)
    model_lr.fit(X_train, y_train)
    predictions_lr = model_lr.predict(X_valid)
    accuracy_lr = accuracy_score(y_valid, predictions_lr)
    if accuracy_lr > best_accuracy_lr:
        best_max_iter_lr = max_iter
        best_accuracy_lr = accuracy_lr 

D:\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
D:\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_resu

D:\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
D:\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_resu

D:\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
D:\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_resu

D:\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
D:\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_resu

Wall time: 15.3 s


Выделим параметры лучшей модели Логистической регрессии.

In [13]:
print('Лучшее число итерраций', best_max_iter_lr,'Лучшая точность', best_accuracy_lr)   

Лучшее число итерраций 83 Лучшая точность 0.7356143079315708


Результат, удовлетворяющий условие.

### Выводы по исследованию моделей
Мы исследовали 3 модели (Дерево решений, Случайный лес, Логистическая регрессия), которые показали:

* У дерева решений при глубине в 6, точность наилучшей модели на валидационной выборке равна: 0.79
* У случайного леса при 20 деревьях и глубине 9, точность наилучшей модели на валидационной выборке равна: 0.81
* У Логистической регрессии точность модели на валидационной выборке равна: 0.74

Исходя из этих данных можно сделать вывод, что лучшей моделью является Случайный лес при 20 деревьях с глубиной 9.

## Проверка лучшей модели на тестовой выборке

Проверим модель случайный лес с лучшими гиперпараметрами на тестовой выборке, обучение выполним на выборке 80% (обучающие + валидационные).

In [14]:
model_rfc_test = RandomForestClassifier(
random_state=784, n_estimators=20, max_depth=9, min_samples_split=4, min_samples_leaf=1, criterion='entropy')

model_rfc_test.fit(X, y)
predictions_rfc_test = model_rfc_test.predict(X_test)
accuracy_rfc_test = accuracy_score(y_test, predictions_rfc_test)
print('Точность модели на тестовой выборке:', accuracy_rfc_test)

Точность модели на тестовой выборке: 0.8087091757387247


### Вывод 

Точность модели случайный лес не изменилась.
Точность на валидационной выборке: 0.81
Точность на тестовой выборке: 0.81

## Проверка модели на адекватность

Оценить адекватность модели можно используя несколько метрик:

* accuracy - точность совпадения прогнозов с правильными ответами.
* precision - отношение правильно предсказанных классов равных 1(TP) к сумме правильно предсказанных классов равных 1(TP) с неправильными предсказаниями, указанные как 1(FP).
* recall - отношение правильно предсказанных классов равных 1(TP) к сумме правильно предсказанных классов равных 1(TP) с неправильными предсказаниями, указанные как 0(FN).

In [15]:
test_predictions = model_rfc_test.predict(X_test)
accuracy = accuracy_score(y_test, test_predictions)
precision = precision_score(y_test, test_predictions)
recall = recall_score(y_test, test_predictions)
print('Accuracy =', accuracy, 'Precision =', precision, 'Recall =', recall)

Accuracy = 0.8087091757387247 Precision = 0.7777777777777778 Recall = 0.5077720207253886


Для оценки адекватности используется F-мера - среднее гармоническое между precision и recall. Если хотя бы один из параметров близок к нулю, то и F-мера стремится к 0. Если оба стремятся к 1, то F-мера тоже стремится к 1.

In [16]:
f_score = f1_score(y_test, test_predictions)
print('F-мера =', f_score)

F-мера = 0.6144200626959249


### Вывод 
Адекватность модели оценивается по нескольким параметрам. Для модели классификации это accuracy, precision и recall. Качество модели лучше всего отражают precision и recall и эти метрики складываются в F-мера как единую оценку модели.

Модель получилась удовлетворяющего качества. И для таких задач она адекватна.

## Итоговый вывод

В ходе проделанной работы было выполнено:

* Изучен файл с данными.
* Разбиты данные на три выборки: обучающая, валидационная и тестовая.
* Исследованы три модели классификации: Решающее дерево, Случайный лес и Логистическая регрессия.
* Найдены оптимальные параметры для каждой модели и выбрана одна из них для обучения модели.
* Оценена точность обученной модели.
* Оценена адекватность модели.
* Для обучения модели было использована выборка в 2000 значений. Этого может быть недостаточно для повышения ключевых параметров модели классификатора. Увеличение в 10 или 100 увеличит accuracy, precision и recall, а также время обучения.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75
